### CW_extract_features.ipynb

This is mostly a development notebook to test out some of the functionality. 
E.g. intensity windowing. (It seems that this is very important for the model's performace. Which I only know, because I forgot it in the beginning, and it then did not work at all)

In [ ]:
%load_ext autoreload
%autoreload 2

# imports 
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk

# # # custom imports:
# import cseg_utils.utils
# import cseg_utils.project_specific.hvpg.dataset
# from cseg_utils.project_specific.hvpg.dataset.dataset import load_image_hvpg_data_and_split_info, get_files_dict_by_regex_pattern

import radipop_utils 
import radipop_utils.visualization
import radipop_utils.features
from radipop_utils.features import (
    convert_and_extract_from_nii,
)


# load user/ system specific env variables:
from dotenv import dotenv_values, find_dotenv
config = dotenv_values(find_dotenv())  # load environment variables as dictionary

# path to the data. You will (likely need to change this)
DATA_ROOT_DIRECTORY = Path(config["DATA_ROOT_DIRECTORY"])

import os
from pathlib import Path
import radipop_utils

# needed for loading the settings for the radiomics feature extractor 
path = Path(os.path.abspath(radipop_utils.__file__))
RADIPOP_PACKAGE_ROOT = path.parent.parent


**Notes:** DICOM to NII conversion
We will assume that this was already done. 
If not, you can use the function: `radipop_utils.utils.dcm2nii` as such:

```python
# convert one dicom folder: 
out_path = Path("/path/to/where/you/want/the/nii/files/")
path_input_single = Path("/path/to/dicom/folder")
radipop_utils.utils.dcm2nii(path_input_single, out_path, out_id = "patient12345")
```

We moreover assume that the segementation masks are available. In the case of the RADIPOP manuscript these were generated with a custom segmentation software used by the radiologists. This software is available upon request.

In [ ]:
# create file with paths of masks and images (using nnUNet results)
#
#dataset = "Dataset125_LSS"
#df = load_image_hvpg_data_and_split_info(DATA_ROOT_DIRECTORY, dataset=dataset)

# df.to_excel(RADIPOP_PACKAGE_ROOT / "data" / "file_paths_and_hvpg_data.xlsx")

#df = pd.read_excel(RADIPOP_PACKAGE_ROOT / "data" / "file_paths_and_hvpg_data.xlsx")

In [ ]:
# create file with paths for **downsampled** dataset 

# dataset = "Dataset302"
# DATA_ROOT_DIRECTORY_2 = "/home/cwatzenboeck/data/nnUNet_frame"
# # df = load_image_hvpg_data_and_split_info(DATA_ROOT_DIRECTORY_2, dataset=dataset, use_nnUNet_segmentation=False)

# # # check existence: 
# # for i in range(len(df)): 
# #     p = df.loc[0, "masks"]
# #     assert os.path.isfile(p), f"{p} is not a file/ does not exist"
# #     p = df.loc[0, "images"]
# #     assert os.path.isfile(p), f"{p} is not a file/ does not exist"


# # df.to_excel(RADIPOP_PACKAGE_ROOT / "data" / f"file_paths_and_hvpg_data_{dataset}.xlsx")

# df = pd.read_excel(RADIPOP_PACKAGE_ROOT / "data" / f"file_paths_and_hvpg_data_{dataset}.xlsx")


In [ ]:
#  create file with paths original dataset (as close as possible to Sin et al. )

dataset = "Dataset125_LSS"
# DATA_ROOT_DIRECTORY_2 = "/home/cwatzenboeck/data/nnUNet_frame"
# df = load_image_hvpg_data_and_split_info(DATA_ROOT_DIRECTORY_2, dataset=dataset, use_nnUNet_segmentation=False)

# # # check existence: 
# for i in range(len(df)): 
#     p = df.loc[0, "masks"]
#     assert os.path.isfile(p), f"{p} is not a file/ does not exist"
#     p = df.loc[0, "images"]
#     assert os.path.isfile(p), f"{p} is not a file/ does not exist"


#df.to_excel(RADIPOP_PACKAGE_ROOT / "data" / f"file_paths_and_hvpg_data_{dataset}_no_autoseg.xlsx")
df = pd.read_excel(RADIPOP_PACKAGE_ROOT / "data" / f"file_paths_and_hvpg_data_{dataset}_no_autoseg.xlsx")



In [ ]:
# filter out the healthy patients where we do not have HVPG data:
m1 = df["set type"] == "external Ts"
m2 = df[~m1]["id"].apply(lambda x: int(x.split("patient")[-1])) < 1000
m = m1 | m2

df_true = df[m]
df_true.to_excel(RADIPOP_PACKAGE_ROOT / "data" / f"file_paths_and_hvpg_data_{dataset}_only_real_data.xlsx")

In [ ]:
# list(df.loc[0, ["images", "masks"]])

In [ ]:

# RADIOMICS_DIR = DATA_ROOT_DIRECTORY / "radiomics" / dataset / "radipop"
# os.makedirs(RADIOMICS_DIR, exist_ok=True)


Let's visualize how our masks /cts look like. This is of course optional

In [ ]:
# # load image and plot overlayed masks + image for one slice:
# note that this has a bad contrast. It is better to use some windowing. 

image_loc = df.loc[0, "images"]
mask_loc = df.loc[0, "masks"]
patientid = df.loc[0, "id"]

mask = sitk.ReadImage(mask_loc)
img = sitk.ReadImage(image_loc)

img_array = sitk.GetArrayFromImage(img)
mask_array = sitk.GetArrayFromImage(mask)
print(img_array.shape, mask_array.shape)

idx = 120
image_xy_slice = img_array[idx, :, :]
label_xy_slice = mask_array[idx, :, :]

crop = lambda x: radipop_utils.visualization.crop_image(x, top_ratio=0.15, bottom_ratio=0.2, left_ratio=0.05, right_ratio=0.02)
image_xy_slice = crop(image_xy_slice)
label_xy_slice = crop(label_xy_slice)

color_dictionary = {
    'violet': (170/255, 48/255, 127/255, 0.35),
    'blue':   (0, 0, 1, 0.35),
}
#                                         ct               liver                        spleen                      
radipop_utils.visualization.overlay_masks(image_xy_slice, [label_xy_slice==1,           label_xy_slice==2], 
                                                          [color_dictionary['violet'],  color_dictionary['blue']]);
radipop_utils.visualization.overlay_masks(image_xy_slice, [], []);

In [ ]:
# Perform some windowing on the image: 
# a typical soft tissue window is  
# wl=50, ww=500
# in the paper a slightly different one was used:
# wl=60, ww=400 

# fe_setting = RADIPOP_PACKAGE_ROOT / "yaml" / "exampleCT.yaml" # not needed ... 


img_transformed = convert_and_extract_from_nii(image_loc, out_range=[0, 255], wl=60, ww=400, dtype=np.uint8)  

img_array = sitk.GetArrayFromImage(img_transformed)
mask_array = sitk.GetArrayFromImage(mask)

idx = 120
image_xy_slice = img_array[idx, :, :]
label_xy_slice = mask_array[idx, :, :]

crop = lambda x: radipop_utils.visualization.crop_image(x, top_ratio=0.15, bottom_ratio=0.2, left_ratio=0.05, right_ratio=0.02)
image_xy_slice = crop(image_xy_slice)
label_xy_slice = crop(label_xy_slice)

color_dictionary = {
    'violet': (170/255, 48/255, 127/255, 0.35),
    'blue':   (0, 0, 1, 0.35),
}
#                                         ct               liver                        spleen                      
radipop_utils.visualization.overlay_masks(image_xy_slice, [label_xy_slice==1,           label_xy_slice==2], 
                                                          [color_dictionary['violet'],  color_dictionary['blue']]);

radipop_utils.visualization.overlay_masks(image_xy_slice, [], []);